In [2]:
import time
from logging import getLogger

from memory_profiler import profile
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.general_recommender import BPR, LINE, NeuMF, ConvNCF, DMF, ItemKNN, NAIS, SpectralCF, GCMC, LightGCN, \
    DGCF, MultiVAE, MacridVAE, CDAE, RaCT, RecVAE, EASE, SLIMElastic, NCL, DiffRec
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger
import copy



In [6]:

if __name__ == '__main__':
    parameter_dict = {
        'metrics': ['Recall', 'Precision', 'GAUC', 'MRR', 'NDCG', 'Hit', 'MAP',  'AveragePopularity', 'GiniIndex', 'ShannonEntropy'],
  #      'epochs':1
    }
    # configurations initialization
    config = Config(model='BPR', dataset='ml-1m', config_dict=parameter_dict)


    # init random seed
    init_seed(config['seed'], config['reproducibility'])

    # logger initialization
    init_logger(config)
    logger = getLogger()

    # write config info into log
    logger.info(config)

    # dataset creating and filtering
    dataset = create_dataset(config)
    logger.info(dataset)

    # dataset splitting
    train_data, valid_data, test_data = data_preparation(config, dataset)

    # model loading and initialization
    model = BPR(config, train_data.dataset).to(config['device'])
    logger.info(model)

    # trainer loading and initialization
    trainer = Trainer(config, model)


    t_s = time.time()
    # model training
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)
    t_f = time.time()
    test_result = trainer.evaluate(test_data)
    t_e = time.time()
    print("Fit -> ", t_f-t_s, "Predict -> ", t_e-t_f)

    @profile
    def foo():
        a = copy.deepcopy(model)
    foo()

    print(test_result)


31 Jan 03:40    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/ml-1m
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'Precision', 'GAUC', 'MRR', 'NDCG', 'Hit', 'MAP', 'AveragePopularity', 'GiniIndex', 'ShannonEntropy']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_

Fit ->  522.7695097923279 Predict ->  7.628895998001099
ERROR: Could not find file /tmp/ipykernel_231/1377057238.py
OrderedDict([('recall@10', 0.1625), ('precision@10', 0.1998), ('gauc', 0.9266), ('mrr@10', 0.4452), ('ndcg@10', 0.2556), ('hit@10', 0.7424), ('map@10', 0.1517), ('averagepopularity@10', 1147.649), ('giniindex@10', 0.8849), ('shannonentropy@10', 0.0035)])
